# PHYS 3250 - Spring with Cart and Pendulum (Taylor 7.31)

From the Lagrangian and using Sympy, the equations of motion were determined to be

$$
    \ddot{x} =
    \frac{M g \sin(2\phi)/2 + M l \dot{\phi}^2 \sin \phi - k x}
    {m + M\sin^2 \phi}
$$

and

$$
    \ddot{\phi} = \frac{k x \cos \phi - (m+M)g\sin \phi - M l \dot{\phi}^2 \sin(2\phi)}
    {\left( m + M \sin^2 \phi \right)l}
$$

## Imports

In [ ]:
%matplotlib widget

In [ ]:
# Numerical
import numpy as np
from scipy.integrate import solve_ivp as sivp
from numba import njit

# Plotting
from matplotlib import pyplot as plt
from matplotlib import patches
from matplotlib import animation

In [ ]:
plt.rcParams["animation.html"] = "jshtml"
_ = plt.ioff()

## Functions

In [ ]:
# System Equation of Motion
@njit(nogil=True)
def eom(t, X, m, M, g, k, L):

    # Unpack Inputs
    x, xD, ph, phD = X

    # Get sin phi
    sinPhi  = np.sin(ph)
    sin2Phi = np.sin(2*ph)

    # Get Numerator LCM
    mu = m + M*sinPhi**2

    # Get x double dot
    xDD  = M*g*sin2Phi/2 + M*L*phD**2*sinPhi - k*x
    xDD /= mu

    # Get phi double dot
    phDD  = k*x*np.cos(ph) - (m+M)*g*sinPhi - M*L*phD**2*sin2Phi/2
    phDD /= mu*L

    # Return
    return [xD, xDD, phD, phDD]

# Jacobian
@njit(nogil=True)
def jac(t, X, m, M, g, k, L):

    # Unpack Inputs
    x, xD, ph, phD = X

    # Get Commonly Used Terms
    sinPhi  = np.sin(ph)
    sin2Phi = np.sin(2*ph)
    sinPhi2 = sinPhi**2
    cosPhi  = np.cos(ph)
    cos2Phi = np.cos(2*ph)
    cosPhi2 = cosPhi**2
    phD2    = phD**2
    
    # Get Numerator LCM
    mu = m + M*sinPhi**2

    # Indiv Pieces
    dxDDdPhi  = L*M*phD2/4*(np.cos(3*ph) - cosPhi)
    dxDDdPhi += L*m*cosPhi*phD2
    dxDDdPhi += M*g*(cos2Phi - 1)/2
    dxDDdPhi += m*g*cos2Phi
    dxDDdPhi += k*x*sin2Phi
    dxDDdPhi *= M/mu**2

    dpDDdPhi  = M**2*L*phD2*sinPhi2
    dpDDdPhi += M*m*L*phD2*(1 - 2*cosPhi2)
    dpDDdPhi += M*g*cosPhi*(M*sinPhi2 - m*cosPhi2)
    dpDDdPhi -= m**2*g*cosPhi
    dpDDdPhi -= (M+m)*k*x*sinPhi
    dpDDdPhi -= m**2*g*cosPhi
    dpDDdPhi /= L*mu**2

    # Setup the Output
    out = np.zeros((4, 4))
    out[0, 1] = 1
    out[1, 0] = -k/mu
    out[1, 2] = dxDDdPhi
    out[1, 3] = 2*M*L*sinPhi*phD/mu
    out[2, 3] = 1
    out[3, 0] = k*cosPhi/(mu*L)
    out[3, 2] = dpDDdPhi
    out[3, 3] = -2*M*sin2Phi*phD/(M+2*m-M*np.cos(2*ph))

    # Return
    return out

## Simulation

The following cells must be run in sequence. That is, if the setup changes, all of the subsequent cells must also be run.

### Setup

In [ ]:
# Close other open plots
plt.close('all')

# Establish Variables
massCart = 0.5
massBob  = 0.05
g        = 9.8
k        = 10.
L        = 0.5

# Set the Solution Time
nTime = 1001
t0 = 0.
tF = 20.
t_span = [t0, tF]
t_eval = np.linspace(t0, tF, nTime)
dt = t_eval[1] - t_eval[0]

# Initial Conditions
x0   = -1.
xD0  =  0.
ph0  =  0.
phD0 =  0.
y0 = [x0, xD0, ph0, phD0]

# Rename according to equations
m = massCart
M = massBob

### Solve the ODE

In [ ]:
# Get the solution
sol = sivp(eom, t_span, y0, args=(m, M, g, k, L), t_eval=t_eval, method='LSODA', jac=jac)

### Plot Results

In [ ]:
# Get Figure
fig, ax = plt.subplots(2, 2, figsize=(9, 9))

# Plot Results
_ = ax[0, 0].plot( # x wrt t
    sol.t, sol.y[0]
)
_ = ax[0, 1].plot( # xD wrt x
    sol.y[0], sol.y[1]
)
_ = ax[1, 0].plot( # phi wrt t
    sol.t, sol.y[2]
)
_ = ax[1, 1].plot( # phD wrt ph
    sol.y[2], sol.y[3]
)

# Labels
# TL: x wrt t
_ = ax[0, 0].set_xlabel('$t$ (s)')
_ = ax[0, 0].set_ylabel('$x$ (m)')
# TR: xD wrt x
_ = ax[0, 1].set_xlabel('$x$ (m)')
_ = ax[0, 1].set_ylabel('$\dot{x}$ (m/s)')
# TL: phi wrt t
_ = ax[1, 0].set_xlabel('$t$ (s)')
_ = ax[1, 0].set_ylabel('$\phi$ (rad)')
# TR: xD wrt x
_ = ax[1, 1].set_xlabel('$\phi$ (rad)')
_ = ax[1, 1].set_ylabel('$\dot{\phi}$ (rad/s)')

# Limits
_ = ax[0, 0].set_xlim(t0, tF)
_ = ax[1, 0].set_xlim(t0, tF)

# Change 
_ = fig.tight_layout()

# Show
plt.show()

# Save
fig.savefig('Spring-Cart-Pendulum-ParameterPlots.png')

### Make Animation

In [ ]:
# Cart Size
cartLen = 0.2
cartHei = 0.1
bobSz   = 16

# Other Important Plot Things
xMin, xMax = -1.25*abs(x0), 1.25*abs(x0)
yMin, yMax = -1.1*L, 2*cartHei
wid  =  xMax - xMin
hei  =  yMax - yMin

# Spring Argument
sprRad     = cartHei / 2
nTurns     = 40
nSpringPts = 501
ptPad      = 40
sprLen     = sol.y[0][0] - xMin
w          = np.linspace(0, sprLen, nSpringPts)
xSpr       = w + xMin
ySpr       = sprRad*np.ones_like(w)
ySpr[ptPad:-ptPad] = sprRad * (1 + np.sin(2*np.pi * nTurns * w[ptPad:-ptPad] / sprLen)/2)

# Initialize Figure & Axes
fig, ax = plt.subplots(figsize=(9, 1.25*9*hei/wid))

# Initialize Artists
cart = ax.add_patch(
    patches.Rectangle((sol.y[0][0], 0), cartLen, cartHei, color='k', zorder=0)
)
string = ax.plot(
    [cartLen/2+sol.y[0][0], cartLen/2+sol.y[0][0] + L*np.sin(sol.y[2][0])],
    [cartHei/2, cartHei/2 - L*np.cos(sol.y[2][0])],
    zorder=5
)[0]
bob = ax.plot(
    cartLen/2+sol.y[0][0] + L*np.sin(sol.y[2][0]),
    cartHei/2 - L*np.cos(sol.y[2][0]),
    linestyle=None, marker='o', ms=bobSz,
    zorder=10
)[0]
spring = ax.plot(
    xSpr, ySpr,
    c='k', linewidth=0.75, zorder=-5
)[0]
_ = ax.hlines(0, xMin, xMax, colors='k')

# Initial Limits
_ = ax.set_xlim(xMin, xMax)
_ = ax.set_ylim(-1.1*L, 2*cartHei)

# Tight
_ = fig.tight_layout()

# Set Aspect Ratio
_ = ax.set_aspect('equal')

# Animation Update Function
def update(frame):

    # Update Cart
    cart.set_x(sol.y[0][frame])

    # Update String
    string.set_data([
        [cartLen/2+sol.y[0][frame], cartLen/2+sol.y[0][frame] + L*np.sin(sol.y[2][frame])],
        [cartHei/2, cartHei/2 - L*np.cos(sol.y[2][frame])]
    ])

    # Update bob
    bob.set_data(
        [cartLen/2+sol.y[0][frame] + L*np.sin(sol.y[2][frame])],
        [cartHei/2 - L*np.cos(sol.y[2][frame])]
    )

    # Update Spring
    sprLen     = sol.y[0][frame] - xMin
    w          = np.linspace(0, sprLen, nSpringPts)
    xSpr       = w + xMin
    spring.set_xdata(xSpr)

# Create the Animation
ani = animation.FuncAnimation(fig=fig, func=update, frames=nTime, interval=dt)
ani

In [ ]:
# Save the Animiation
_ = ani.save(filename="Spring-Cart-Pendulum-Animation.mp4", writer="ffmpeg", fps=1/dt)

In [ ]:
# Close Animation Figure
plt.close(fig)